In [31]:
import pysftp
import os 
import numpy as np
import pandas as pd
import netCDF4
import numpy as np

from scipy import io
from classes.matlab_reader import matlab_reader
from classes.grid_funcs import map_grid,dataset,data_wizard
from dotenv import load_dotenv   #for python-dotenv method
from datetime import datetime

pd.options.mode.chained_assignment = None  # default='warn'

ssw_list = [datetime.strptime(file_name[:10], '%Y-%m-%d') for file_name in os.listdir('ssw_events_saved/')]
ssw_list.sort()

In [32]:
file = '2002-10-10_mfy.npz'
# get only the date from the file name
file_date = file[:10].strip('_')

In [33]:

def generate_dataset(path = 'processed_airs_sftp/mfy/', col_names ='col_name'):
    # path: path to the folder containing the npz files
    # col_names: name of the column in the dataframe
    date_col = []
    data_mean_col = []
    for file in os.listdir(path):
        # print(file)
        if file != ".DS_Store":
            data = np.load(path + file, allow_pickle=True)['arr_0']
            data_mean = np.nanmean(data)
            file_date = file[:10]
            date_col.append(file_date)
            data_mean_col.append(data_mean)

    df = pd.DataFrame(data={
        'dates' : date_col,
        f'{col_names}_data_mean' : data_mean_col})
    # add description of the data
    df['dates'] = pd.to_datetime(df['dates'])
    df = df.sort_values(by='dates')
    df = df.reset_index()
    df = df.drop(columns=['index'])
    df['year'] = df['dates'].apply(lambda x: x.year)

    return df

df_mfy = generate_dataset(path = 'processed_airs_sftp/mfy/',col_names='mfy')
df_mfx = generate_dataset(path = 'processed_airs_sftp/mfx/',col_names='mfx')
df_bg = generate_dataset(path = 'processed_airs_sftp/bg/',col_names='bg')


/var/folders/p6/325r0tsd2cd1jddpt9nd70hm0000gn/T/ipykernel_89404/298112681.py:10: RuntimeWarning:

Mean of empty slice

/var/folders/p6/325r0tsd2cd1jddpt9nd70hm0000gn/T/ipykernel_89404/298112681.py:10: RuntimeWarning:

Mean of empty slice

/var/folders/p6/325r0tsd2cd1jddpt9nd70hm0000gn/T/ipykernel_89404/298112681.py:10: RuntimeWarning:

Mean of empty slice



In [34]:
df_merged = pd.merge(df_mfx, df_mfy, how = 'left', on = 'dates')
df_merged = pd.merge(df_merged, df_bg, how = 'left', on = 'dates')
# df_merged[['mfy_data_mean']] = df_merged[['mfy_data_mean']] * np.cos(np.deg2rad(60))
df_merged['dayofyear'] = df_merged['dates'].dt.dayofyear
# multiply square root of the columns mfx_data_mean squared and mfy_data_mean squared in the df_all dataframe
df_merged['mf_abs'] = np.sqrt(df_merged['mfx_data_mean']**2 + df_merged['mfy_data_mean']**2)


In [35]:
# save df_merged to csv file
df_merged.to_csv('df_merged.csv', index=False)

In [36]:
import plotly.graph_objects as go
df_all  = df_merged

fig = go.Figure()
fig.add_shape(type='line', x0=df_all['dates'].min(), y0=0, x1=df_all['dates'].max(), y1=0, line=dict(color='gray', dash='dash'))

for year, df in df_all.groupby('year'):
    fig.add_trace(go.Scatter(x=df['dayofyear'], y=df['mfx_data_mean'], name=f'{year}', line=dict()))

    # Set layout for each year
    fig.update_layout(
        yaxis=dict(
            range=[-0.002, 0.0005]),
        xaxis=dict(title='Dates'),
        title=f'Momentum Flux (x) in listed years',
        legend=dict(orientation="h", y=1.2, x=0.5)
    )

    # Show the plot for each year
fig.show()

In [37]:
df_all.head(3)

dates  mfx_data_mean  year_x  mfy_data_mean  year_y  bg_data_mean  \
0 2002-08-31      -0.000049    2002      -0.000066    2002    225.517001   
1 2002-09-01       0.000035    2002      -0.000093    2002    225.224340   
2 2002-09-02       0.000043    2002      -0.000115    2002    225.338779   

   year  dayofyear    mf_abs  
0  2002        243  0.000082  
1  2002        244  0.000100  
2  2002        245  0.000123

In [38]:
df_ssw = pd.read_csv('ssw_events_saved/2018-02-11_ssw.csv')

In [39]:
df_ssw['dates'] = pd.to_datetime(df_ssw['dates'])

In [40]:
df_ssw_merged = pd.merge(df_ssw,df_all[['dates','mfx_data_mean','mfy_data_mean','bg_data_mean','dayofyear']],how='left', on='dates')

In [41]:
df_ssw_merged.head(5)

dates  wind_eastward_mean  wind_eastward_mean_selection  \
0 2018-01-12           36.802911                     35.894021   
1 2018-01-13           35.775050                     45.111992   
2 2018-01-14           33.788233                     53.774738   
3 2018-01-15           30.535844                     64.858844   
4 2018-01-16           25.154227                     75.345086   

   temperature_mean_selection  wind_eastward_mean_cos  temperate_mean_cos  \
0                  193.407067               18.401455           96.703533   
1                  192.508725               17.887525           96.254362   
2                  192.749715               16.894116           96.374857   
3                  198.135240               15.267922           99.067620   
4                  203.096840               12.577114          101.548420   

   temperature_pct_change   SSW_  SSW_flag  mfx_data_mean  mfy_data_mean  \
0               -0.130334  False     False      -0.005338      -0.000139   
1               -0.464483  False     False      -0.008727      -0.000898   
2                0.125184  False     False      -0.005805      -0.001071   
3                2.794051  False     False      -0.007918      -0.003716   
4                2.504148  False     False      -0.004507      -0.001491   

   bg_data_mean  dayofyear  
0    209.865340       12.0  
1    210.253968       13.0  
2    210.937761       14.0  
3    210.422150       15.0  
4    197.550383       16.0

In [42]:
df['mfy_data_mean'] = df['mfy_data_mean'].interpolate(method='polynomial', order=2)
df['dayofyear'] = df['dayofyear'].interpolate(method='polynomial', order=2)

In [43]:
import plotly.graph_objects as go

df = df_ssw_merged
fig = go.Figure()

fig.add_trace(go.Scatter(x=df['dates'], y=df['wind_eastward_mean_cos'], name='wind_eastward_mean_selection', line=dict(color='blue')))
# fig.add_trace(go.Scatter(x=df['dates'], y=df['temperature_mean_selection'], name='Temperature Selection Mean', line=dict(), yaxis='y2'))
fig.add_trace(go.Scatter(x=df['dates'], y=df['mfy_data_mean'], name='mfy_data_mean', line=dict(color='orange'), yaxis='y2'))

# Add a dashed line at y=0
fig.add_shape(type='line', x0=df['dates'].min(), y0=0, x1=df['dates'].max(), y1=0, line=dict(color='gray', dash='dash'))

# Find the index of the first negative value
first_negative_index = (df['wind_eastward_mean_cos'] < 0).idxmin()

# Add vertical lines for SSW_flag == True
for i, row in df.iterrows():
    if row.SSW_flag == True:
        fig.add_shape(type='line', x0=row.dates, y0=-50, x1=row.dates, y1=50, line=dict(color='red', dash='dash'))

# Set layout
fig.update_layout(
    yaxis=dict(
        range=[-10, 30],
        title='wind_eastward_mean_cos'
    ),
    yaxis2=dict(
        range=[-0.001, 0.001],
        title='mfy_data_mean',
        side='right',
        overlaying='y'
    )
    ,xaxis=dict(title='Dates'),
    title=f'ZMZW at 60◦ N latitude',
    legend=dict(orientation="h", y=1.2, x=0.5)
)

# Show the plot
fig.show()



In [44]:
grid = map_grid(grid_size=125, source='airs')
wizard = data_wizard()

temp_data = np.load('processed_airs_sftp/mfy/2002-08-31_mfy.npz')['arr_0']
    
select_area_indexer = grid.select_area_indexer(
    min_lat= 55,
    max_lat= 65,
    min_lon= 50,
    max_lon= 70)

data = wizard.data_filtering(select_area_indexer = select_area_indexer ,data = temp_data)

In [45]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap

fig = plt.figure(figsize=(10, 8))

# use to plot areas in any section of the globe using a stereographic projection
# m = Basemap(projection='stere', resolution='c',
#             width=8E6, height=8E6, 
#             lat_0=55, lat_1=65,
#             lon_0=0,lon_1 = 360)

# use to plot north pole stero plot
m = Basemap(projection='npstere',boundinglat=0,lon_0=0,resolution='c')
m.shadedrelief(scale=0.5)

m.pcolormesh(grid.nh_lon, 
             grid.nh_lat, 
             data,
             latlon=True, cmap='RdBu_r',
             shading='auto')

m.drawcoastlines(linewidth=0.5,linestyle='solid')
m.drawcountries(linewidth=0.5, linestyle='solid',color='k')

plt.title('Temperature')
plt.colorbar(label='Temperature in (°C)')

/var/folders/p6/325r0tsd2cd1jddpt9nd70hm0000gn/T/ipykernel_89404/569046933.py:18: UserWarning:

The input coordinates to pcolormesh are interpreted as cell centers, but are not monotonically increasing or decreasing. This may lead to incorrectly calculated cell edges, in which case, please supply explicit cell edges to pcolormesh.

